In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

# Week 1

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Initial data ---
#X_init = np.array([...])  # your 15x3 input array
#y_init = np.array([...])  # your 15 outputs

X_init = datain
y_init = dataout

In [ ]:
# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because minimize()

# --- Optimize acquisition function with multiple random starts ---
y_best = y_init.max()
bounds = [(0,1), (0,1), (0,1)]

best_x = None
best_ei = float('inf')
for _ in range(10):
    x0 = np.random.rand(3)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next compound combination to try:", x_next)


#Week 2

In [ ]:
import numpy as np

# Add the new observation
X_new = np.array([[0.901884, 0.273619, 0.723590]])
y_new = np.array([-0.13917950276182944])

# Combine with previous data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

X_init = X_all
y_init = y_all

print(X_init)
print(y_init)


In [ ]:
# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

In [ ]:
# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because minimize()

In [ ]:
# --- Optimize acquisition function with multiple random starts ---
y_best = y_init.max()
bounds = [(0,1), (0,1), (0,1)]

best_x = None
best_ei = float('inf')
for _ in range(30):
    x0 = np.random.rand(3)
    res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.01),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next compound combination to try:", x_next)

# Week 3

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([[0.17152521, 0.34391687, 0.2487372 ],
                   [0.24211446, 0.64407427, 0.27243281],
                   [0.53490572, 0.39850092, 0.17338873],
                   [0.49258141, 0.61159319, 0.34017639],
                   [0.13462167, 0.21991724, 0.45820622],
                   [0.34552327, 0.94135983, 0.26936348],
                   [0.15183663, 0.43999062, 0.99088187],
                   [0.64550284, 0.39714294, 0.91977134],
                   [0.74691195, 0.28419631, 0.22629985],
                   [0.17047699, 0.6970324 , 0.14916943],
                   [0.22054934, 0.29782524, 0.34355534],
                   [0.66601366, 0.67198515, 0.2462953 ],
                   [0.04680895, 0.23136024, 0.77061759],
                   [0.60009728, 0.72513573, 0.06608864],
                   [0.96599485, 0.86111969, 0.56682913]])
y_init = np.array([-0.1121222 , -0.08796286, -0.11141465, -0.03483531, -0.04800758,
                   -0.11062091, -0.39892551, -0.11386851, -0.13146061, -0.09418956,
                   -0.04694741, -0.10596504, -0.11804826, -0.03637783, -0.05675837])

# --- Add last week's observation ---
X_init = np.vstack([X_init, [0.901884, 0.273619, 0.723590]])
y_init = np.concatenate([y_init, [-0.13917950276182944]])

# --- Add this week's observation ---
X_init = np.vstack([X_init, [0.393393, 0.209704, 0.929527]])
y_init = np.concatenate([y_init, [-0.1691006942213739]])

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because minimize()

# --- Optimize acquisition function ---
bounds = [(0,1), (0,1), (0,1)]
y_best = y_init.max()

best_x = None
best_ei = float('inf')
for _ in range(50):
    x0 = np.random.rand(3)
    res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.01),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
x_next_6dp = np.round(x_next, 6)
x_next_6dp

print("Next compound combination to try:", x_next)

In [ ]:
print("Next compound combination to try:", x_next_6dp)

# Week 4

New point: [0.117994, 0.128418, 0.478883]
Output: -0.04303671
This new point’s output (-0.043) is slightly worse than the best so far, but better than many other points
Exploration is yielding useful information: 
The GP now knows that very high or extreme values of compounds are likely to produce higher adverse reactions
so it can shift focus toward more moderate combinations.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([
    [0.17152521, 0.34391687, 0.2487372 ],
    [0.24211446, 0.64407427, 0.27243281],
    [0.53490572, 0.39850092, 0.17338873],
    [0.49258141, 0.61159319, 0.34017639],
    [0.13462167, 0.21991724, 0.45820622],
    [0.34552327, 0.94135983, 0.26936348],
    [0.15183663, 0.43999062, 0.99088187],
    [0.64550284, 0.39714294, 0.91977134],
    [0.74691195, 0.28419631, 0.22629985],
    [0.17047699, 0.6970324 , 0.14916943],
    [0.22054934, 0.29782524, 0.34355534],
    [0.66601366, 0.67198515, 0.2462953 ],
    [0.04680895, 0.23136024, 0.77061759],
    [0.60009728, 0.72513573, 0.06608864],
    [0.96599485, 0.86111969, 0.56682913]
])

y_init = np.array([
    -0.1121222 , -0.08796286, -0.11141465, -0.03483531, -0.04800758,
    -0.11062091, -0.39892551, -0.11386851, -0.13146061, -0.09418956,
    -0.04694741, -0.10596504, -0.11804826, -0.03637783, -0.05675837
])

# --- Add new observations from previous rounds ---
X_new = np.array([
    [0.901884, 0.273619, 0.723590],
    [0.393393, 0.209704, 0.929527],
    [0.117994, 0.128418, 0.478883]
])

y_new = np.array([
    -0.13917950276182944,
    -0.1691006942213739,
    -0.043036710546582216
])

# Combine all data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_all, y_all)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # minimize

# --- Optimize acquisition function ---
bounds = [(0, 1), (0, 1), (0, 1)]
y_best = y_all.max()

best_x, best_ei = None, float('inf')
for _ in range(50):  # multiple random restarts
    x0 = np.random.rand(3)
    res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.01),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei, best_x = res.fun, res.x

x_next = np.round(best_x, 6)
print("Next compound combination to try:", x_next)
